In [6]:
import requests,shutil
from io import BytesIO
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
from pyproj import Proj, transform
from osgeo import gdal,osr
from shapely.geometry import shape

In [2]:
url = "http://gdis.mgb.gov.ph/arcgis/rest/services/Flood_Landslide_Susceptibility_10K_new_web/MapServer/export?dpi=96&transparent=true&format=png8&bbox=13768453.20379189%2C1138715.818918207%2C13779699.001578331%2C1147563.404942204&bboxSR=102100&imageSR=102100&size=1177%2C926&f=image"

In [ ]:
url.split('bbox=')[1].split('&bboxSR')[0].split('%2C')

In [3]:
# Download png
impath = "file.png"
r = requests.get(url,stream=True)
# with open(impath,'wb') as im:
#     r.raw.decode_content = True
#     shutil.copyfileobj(r.raw, im)
#     for chunk in r.iter_content(1024):
#         im.write(chunk)

In [8]:
def reproj(x,y):
    inProj = Proj(init='epsg:3857')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj,outProj,x,y)
    return x2,y2

In [9]:
# Get bbounds
def getBounds(url):
    bbox = url.split('bbox=')[1].split('&bboxSR')[0].split('%2C')
    print(bbox)
    to_float = list(map(lambda x: float(x),bbox))
    bbox = []
    for i in [0,2]:
        x,y = reproj(to_float[i],to_float[i+1])
        bbox.extend([x,y])
    return bbox
bounds = getBounds(url)
print(bounds)

['13768453.20379189', '1138715.818918207', '13779699.001578331', '1147563.404942204']
[123.68411951650647, 10.175345268610686, 123.78514223684324, 10.253564805628109]


In [12]:
def arrayToRaster(array,fileName,EPSGCode,xMin,xMax,yMin,yMax,numBands):
    xPixels = array.shape[1]  # number of pixels in x
    yPixels = array.shape[0]  # number of pixels in y
    pixelXSize =(xMax-xMin)/xPixels # size of the pixel in X direction     
    pixelYSize = -(yMax-yMin)/yPixels # size of the pixel in Y direction

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(fileName,xPixels,yPixels,numBands,gdal.GDT_Byte, options = [ 'PHOTOMETRIC=RGB' ])
    dataset.SetGeoTransform((xMin,pixelXSize,0,yMax,0,pixelYSize))  

    datasetSRS = osr.SpatialReference()
    datasetSRS.ImportFromEPSG(EPSGCode)
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    if numBands > 1:
        for i in range(0,numBands):
            dataset.GetRasterBand(i+1).WriteArray(array[:,:,i])
    else:
        dataset.GetRasterBand(1).WriteArray(array)

    dataset.FlushCache()  # Write to disk.

In [13]:
# Read array into function
impath = 'file.png'
img = np.asarray(Image.open(impath))
imgg = mpimg.imread(impath)
fname = "OUTPUT.TIF"
epsg = 4326
xmin,ymin,xmax,ymax = bounds[0],bounds[1],bounds[2],bounds[3]
bands = 1
print(xmin,xmax,ymin,ymax)
arrayToRaster(img,fname,epsg,xmin,xmax,ymin,ymax,bands)


123.68411951650647 123.78514223684324 10.175345268610686 10.253564805628109


In [23]:
# To shp
fname = 'OUTPUT.TIF'
import rasterio
from rasterio.features import shapes
mask = None
# with rasterio.drivers():
with rasterio.open(fname) as src:
    image = src.read(1) # first band
    results = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) 
    in enumerate(
        shapes(image, mask=mask, transform=src.transform)))

In [24]:
geoms = list(results)
len(geoms)

5818

In [26]:
# Filter 0 values
final = []
for geo in geoms:
    rast_val = geo['properties']['raster_val']
    if rast_val != 0.0:
        final.append(geo)

3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0


3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
7.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0


3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
1.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0


[{'properties': {'raster_val': 3.0},
  'geometry': {'type': 'Polygon',
   'coordinates': [[(111.4363588287556, 10.253564805628109),
     (111.4363588287556, 10.376169100910198),
     (111.33991976822213, 10.376169100910198),
     (111.33991976822213, 10.253564805628109),
     (111.4363588287556, 10.253564805628109)]]}},
 {'properties': {'raster_val': 3.0},
  'geometry': {'type': 'Polygon',
   'coordinates': [[(108.83250419435188, 10.253564805628109),
     (108.83250419435188, 10.621377691474379),
     (108.73606513381841, 10.621377691474379),
     (108.73606513381841, 10.253564805628109),
     (108.83250419435188, 10.253564805628109)]]}},
 {'properties': {'raster_val': 3.0},
  'geometry': {'type': 'Polygon',
   'coordinates': [[(94.27020605379771, 10.253564805628109),
     (94.27020605379771, 10.621377691474379),
     (94.17376699326424, 10.621377691474379),
     (94.17376699326424, 10.253564805628109),
     (94.27020605379771, 10.253564805628109)]]}},
 {'properties': {'raster_val': 3.

In [27]:
import geopandas as gp

/home/prinns/anaconda3/envs/pygis/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [28]:
poly  = gp.GeoDataFrame.from_features(final)
poly.crs = {'init' :'epsg:3857'}

In [29]:
# Reproject
poly = poly.to_crs({'init': 'epsg:4326'})
# Save
poly.to_file('output.shp')